## Recuperación ranqueada y vectorización de documentos (RRDV)

Importamos librerías de utilidad.

In [1]:
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd

Importamos nuestras funciones de preprocesamiento de documentos

In [2]:
from documents import Document, load_docs

[nltk_data] Downloading package punkt to /home/gustavo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gustavo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/gustavo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
all_docs = load_docs(Path("./data/docs-raw-texts"))

Definimos la clase para la **Recuperación Ranqueada y Vectorización de Documentos.** Como parte de sus atributos almacena el conteo de términos por documento, y su IDF y TF-IDF. Con estos se hará la búsqueda ranqueada, haciendo uso de la similitud coseno.

In [4]:
class RRDV:
    def __init__(self, docs: List[Document]):
        """
        Inicializa RRDV con una lista de documentos.

        Args:
            docs (List[Document]): Lista de objetos Document para construir el índice TF-IDF.
        """
        self.docs = docs
        
        # Crear un DataFrame con los conteos de términos para cada documento
        self.term_counts = pd.DataFrame({
            doc.name: doc.term_counts for doc in self.docs
        })
        self.term_counts.fillna(0, inplace=True)

        # Calcular la frecuencia de documentos para cada término
        self.document_count = (self.term_counts >= 1).sum(axis=1)

        # Calcular el IDF (Inverse Document Frequency)
        self.idf = np.log10(len(self.docs) / self.document_count)

        # Calcular TF-IDF (Term Frequency-Inverse Document Frequency)
        self.tfidf = np.log10(1 + self.term_counts).mul(self.idf, axis=0)

    @staticmethod
    def cosine_similarity(tfidf_doc_1: pd.Series, tfidf_doc_2: pd.Series | pd.DataFrame) -> pd.Series:
        """
        Calcula la similitud coseno entre dos vectores.

        Args:
            tfidf_doc_1 (pd.Series): Vector del primer documento.
            tfidf_doc_2 (pd.Series | pd.DataFrame): Vector TF-IDF del segundo documento o un DataFrame de vectores.

        Returns:
            pd.Series: Similitudes coseno entre tfidf_doc_1 y tfidf_doc_2.
        """
        return np.dot(tfidf_doc_1, tfidf_doc_2) / (
            np.linalg.norm(tfidf_doc_1) * np.linalg.norm(tfidf_doc_2, axis=0))

    def search(self, query_document: Document, min_similarity: float = 0.0) -> pd.DataFrame:
        """
        Realiza una búsqueda para encontrar documentos similares al documento de consulta.

        Args:
            query_document (Document): Documento de consulta para buscar similitudes.
            min_similarity (float): Umbral mínimo de similitud para filtrar resultados. Por defecto es 0.

        Returns:
            pd.DataFrame: DataFrame con documentos relevantes y sus similitudes.
        """
        # Filtrar términos en el vocabulario del índice
        in_vocab_term_counts = query_document.term_counts[query_document.term_counts.index.isin(self.idf.index)]
        
        # Calcular el TF-IDF para el documento de consulta
        query_tfidf = (np.log10(1 + in_vocab_term_counts) * self.idf).fillna(0)

        # Calcular la similitud coseno entre el documento de consulta y todos los documentos en el índice
        similarities = self.cosine_similarity(query_tfidf, self.tfidf)
        
        # Crear un DataFrame con los resultados de similitud
        results = pd.DataFrame({
            'similarity': similarities,
            'doc': self.docs
        }, index=self.tfidf.columns)
        
        # Ordenar los resultados por similitud de mayor a menor
        results.sort_values(by='similarity', ascending=False, inplace=True)

        # Filtrar los resultados por similitud mínima
        results = results[results['similarity'] > min_similarity]
        
        return results

    def evaluate_search(self, queries: List[Document], output_path: Path):
        """
        Evalúa las consultas y escribe los resultados en un archivo de salida.

        Args:
            queries (List[Document]): Lista de documentos de consulta para evaluar.
            output_path (Path): Ruta del archivo donde se guardarán los resultados.
        """
        with open(output_path, 'w') as output_file:
            for query in queries:
                relevant_docs = self.search(query_document=query)    
                result_texts = [f'{doc_name}:{row.similarity}' for doc_name, row in relevant_docs.iterrows()]
                output_file.write(f"{query.name}\t{','.join(result_texts)}\n")


In [5]:
rrdv = RRDV(all_docs)

In [6]:
all_queries = load_docs(Path("./data/queries-raw-texts"))
rrdv.evaluate_search(all_queries, output_path=Path("./data/RRDV-consultas_resultado"))